# ChromaDB und Embedding-Modelle

In diesem Notebook arbeiten wir mit ChromaDB und vergleichen verschiedene Embedding-Modelle.

In [ ]:
import chromadb
from chromadb.config import Settings
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

## 1. Daten laden

In [ ]:
data_dir = Path('../data')
csv_path = data_dir / 'tagesschau_2023_prepared.csv'

df = pd.read_csv(csv_path)

# Identifiziere Text-Spalte
text_column = None
for col in ['text', 'content', 'article', 'body']:
    if col in df.columns:
        text_column = col
        break

if text_column is None:
    text_column = df.columns[0]

documents = df[text_column].dropna().tolist()
print(f"Geladene Dokumente: {len(documents)}")

# Verwende Stichprobe für Demo
sample_docs = documents[:1000] if len(documents) > 1000 else documents
print(f"Verwendete Dokumente: {len(sample_docs)}")

## 2. ChromaDB initialisieren

In [ ]:
# Erstelle persistente ChromaDB Instanz
chroma_dir = data_dir / 'chromadb'
chroma_dir.mkdir(exist_ok=True)

client = chromadb.PersistentClient(
    path=str(chroma_dir),
    settings=Settings(anonymized_telemetry=False)
)

print(f"ChromaDB Client erstellt")
print(f"Persistenz-Verzeichnis: {chroma_dir}")

## 3. Collection erstellen und Dokumente hinzufügen

In [ ]:
# Erstelle oder lade Collection
collection_name = "tagesschau_articles"

try:
    collection = client.get_collection(name=collection_name)
    print(f"Collection '{collection_name}' geladen")
    print(f"  Anzahl Dokumente: {collection.count()}")
except:
    collection = client.create_collection(name=collection_name)
    print(f"Neue Collection '{collection_name}' erstellt")
    
    # Füge Dokumente hinzu
    print(f"\nFüge {len(sample_docs)} Dokumente hinzu...")
    
    ids = [f"doc_{i}" for i in range(len(sample_docs))]
    metadatas = [{"index": i} for i in range(len(sample_docs))]
    
    collection.add(
        documents=sample_docs,
        ids=ids,
        metadatas=metadatas
    )
    
    print(f"✓ {len(sample_docs)} Dokumente hinzugefügt")
    print(f"  Collection-Größe: {collection.count()}")

## 4. Was können wir sofort beobachten?

ChromaDB hat automatisch:
- Embeddings für alle Dokumente berechnet
- Eine persistente Datenbank erstellt
- Indizierung für schnelle Suche durchgeführt

In [ ]:
# Zeige Collection-Informationen
print(f"Collection-Name: {collection.name}")
print(f"Anzahl Dokumente: {collection.count()}")
print(f"Metadaten: {collection.metadata}")

## 5. Standard-Embedding-Modell von ChromaDB

ChromaDB verwendet standardmäßig **all-MiniLM-L6-v2** von Sentence Transformers.

In [ ]:
# Lade das Standard-Modell
standard_model = SentenceTransformer('all-MiniLM-L6-v2')

print("Standard-Embedding-Modell: all-MiniLM-L6-v2")
print(f"Embedding-Dimension: {standard_model.get_sentence_embedding_dimension()}")
print(f"\nModell-Informationen:")
print(f"  - Multilingual: Nein (nur Englisch)")
print(f"  - Modell-Größe: ~80 MB")
print(f"  - Max Sequence Length: 256 Tokens")
print(f"  - Training: Paraphrase Mining auf 1B+ sentence pairs")

## 6. Suche mit ChromaDB

In [ ]:
def chromadb_search(query, n_results=5):
    """
    Sucht in ChromaDB Collection
    """
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )
    
    return results

# Test-Suchen
test_queries = [
    "Klimawandel",
    "Ukraine Krieg",
    "Bundesregierung"
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"ChromaDB Suche: '{query}'")
    print(f"{'='*60}")
    
    results = chromadb_search(query, n_results=3)
    
    if not results['documents'] or len(results['documents'][0]) == 0:
        print("Keine Ergebnisse gefunden.")
        continue
    
    documents = results['documents'][0]
    distances = results['distances'][0] if 'distances' in results else None
    
    for i, doc in enumerate(documents):
        print(f"\n--- Ergebnis {i+1} ---")
        if distances:
            print(f"Distanz: {distances[i]:.4f}")
        preview = doc[:300] + "..." if len(doc) > 300 else doc
        print(f"Text: {preview}")

## 7. Vergleich verschiedener Embedding-Modelle

In [ ]:
# Modelle zum Vergleich
models_to_compare = [
    ('all-MiniLM-L6-v2', 'ChromaDB Standard (Englisch)'),
    ('paraphrase-multilingual-MiniLM-L12-v2', 'Multilingual MiniLM'),
    ('sentence-transformers/all-mpnet-base-v2', 'MPNet Base (größer, besser)')
]

def compare_models(query, documents_sample, models):
    """
    Vergleicht verschiedene Embedding-Modelle
    """
    print(f"\n{'='*60}")
    print(f"Vergleich von Embedding-Modellen")
    print(f"Query: '{query}'")
    print(f"{'='*60}\n")
    
    results_comparison = {}
    
    for model_name, description in models:
        print(f"\n--- {description} ({model_name}) ---")
        try:
            model = SentenceTransformer(model_name)
            
            # Berechne Embeddings
            query_embedding = model.encode([query])[0]
            doc_embeddings = model.encode(documents_sample)
            
            # Berechne Kosinus-Ähnlichkeit
            similarities = np.dot(doc_embeddings, query_embedding) / (
                np.linalg.norm(doc_embeddings, axis=1) * np.linalg.norm(query_embedding)
            )
            
            top_indices = np.argsort(similarities)[::-1][:5]
            results_comparison[model_name] = [
                (idx, similarities[idx]) for idx in top_indices
            ]
            
            print(f"Top 5 Ergebnisse:")
            for i, (idx, score) in enumerate(results_comparison[model_name], 1):
                print(f"  {i}. Doc {idx}: {score:.4f}")
            
            print(f"  Embedding-Dimension: {model.get_sentence_embedding_dimension()}")
            
        except Exception as e:
            print(f"  Fehler: {e}")
    
    return results_comparison

# Führe Vergleich durch
sample_for_comparison = sample_docs[:50]
comparison_results = compare_models(
    "Klimawandel",
    sample_for_comparison,
    models_to_compare
)

## 8. Modell-Leaderboard

Informationen über Embedding-Modelle finden Sie auf:
- https://huggingface.co/spaces/mteb/leaderboard
- https://www.sbert.net/docs/pretrained_models.html

## 9. Eigenschaften von Embedding-Modelle

### Diskussionspunkte:
1. Welche Eigenschaften kennzeichnen Embedding-Modelle?
2. Welche Probleme könnte es mit dem Standard-Modell bei deutschen Texten geben?
3. Welche Vorverarbeitungsschritte sollten durchgeführt werden?

In [ ]:
# Analysiere mögliche Probleme mit Standard-Modell
print("Mögliche Probleme mit all-MiniLM-L6-v2 für deutsche Texte:")
print("  1. Modell wurde primär auf englischen Texten trainiert")
print("  2. Deutsche Wörter werden möglicherweise nicht optimal dargestellt")
print("  3. Semantische Ähnlichkeiten könnten für deutsche Begriffe ungenau sein")
print("\nLösungsansätze:")
print("  - Verwende multilinguale Modelle (z.B. paraphrase-multilingual-MiniLM-L12-v2)")
print("  - Text-Vorverarbeitung: Normalisierung, Stemming, Lemmatisierung")
print("  - Stopword-Entfernung für deutsche Sprache")
print("  - Entfernung von HTML-Tags und Sonderzeichen")
print("  - Satz-Segmentierung für längere Texte")

## 10. Vorverarbeitung für bessere Ergebnisse

In [ ]:
from bs4 import BeautifulSoup
import re

def preprocess_text(text):
    """
    Vorverarbeitung von Texten
    """
    if pd.isna(text):
        return ""
    
    # Entferne HTML-Tags
    text = BeautifulSoup(text, 'html.parser').get_text()
    
    # Normalisiere Whitespace
    text = re.sub(r'\s+', ' ', text)
    
    # Entferne URLs
    text = re.sub(r'http\S+|www\.\S+', '', text)
    
    # Entferne E-Mail-Adressen
    text = re.sub(r'\S+@\S+', '', text)
    
    # Trim
    text = text.strip()
    
    return text

# Teste Vorverarbeitung
if len(sample_docs) > 0:
    original = sample_docs[0]
    processed = preprocess_text(original)
    
    print("Original (erste 200 Zeichen):")
    print(original[:200])
    print("\nVorverarbeitet (erste 200 Zeichen):")
    print(processed[:200])
    print(f"\nLänge Original: {len(original)}")
    print(f"Länge Vorverarbeitet: {len(processed)}")